# **Funciones para el Funcionamiento del API**

<div style='text-align: justify'>
En este archivo, nos enfocaremos en la creación de las funciones necesarias para el funcionamiento del API. Estas funciones serán responsables de manejar las solicitudes entrantes, procesar los datos según sea necesario y devolver las respuestas adecuadas. Utilizaremos diversas herramientas y bibliotecas, como FastAPI, para crear un API robusto y eficiente que pueda ser utilizado para diversos propósitos, desde la gestión de datos hasta la interacción con modelos de aprendizaje automático. A través de la implementación cuidadosa y la documentación adecuada, garantizaremos que nuestro API sea fácil de entender, utilizar y mantener.

Estas librerias las importaremos despues en el archivo de funciones_basicas
</div>


## 1. Importamos las librerias

In [1]:
import pandas as pd
import funciones_basicas as fb
import warnings
warnings.filterwarnings('ignore')

## 2. Cargamos los datos necesarios para los ejemplos de las funciones

#### Función DEVELOPER

In [5]:
desarrollador = fb.cargar_datos_parquet('PARQUET\TABLAS UNIDAS\DESARROLLADOR.parquet')
desarrollador.head(3)

,PRECIO,ID_GAME,DEVELOPER,AÑO
0,9.99,282010,STAINLESS GAMES LTD,1997
1,9.99,70,VALVE,1998
2,9.99,70,VALVE,1998


In [54]:
def developer(desarrollador: str, dataframe: pd.DataFrame):
    """
    Función que devuelve un DataFrame con información detallada sobre la cantidad de elementos y el contenido gratuito lanzado por un desarrollador en cada año registrado en el DataFrame.

    Parámetros:
        desarrollador (str): El nombre del desarrollador (empresa) del cual se desea obtener información.
        dataframe (pd.DataFrame): El DataFrame que contiene los datos.

    Retorna:
        pd.DataFrame: Un DataFrame con las columnas "AÑO_LANZAMIENTO" (año de lanzamiento), "CANTIDAD DE JUEGOS" (total de elementos lanzados) y "CONTENIDO FREE" (porcentaje de contenido gratuito lanzado).
    """
    listaaños = dataframe['AÑO'].unique()  # Almaceno en una variable los valores únicos del campo Año
    listadeveloper = dataframe['DEVELOPER'].unique()  # Almaceno en una variable los valores únicos del campo developer

    lista = []

    if desarrollador in listadeveloper:
        for años in listaaños:
            todo = len(dataframe[(dataframe['AÑO'] == años) & (dataframe['DEVELOPER'] == desarrollador)])
            cantidaditems = len(dataframe[(dataframe['PRECIO'] == 0) & (dataframe['AÑO'] == años) & (dataframe['DEVELOPER'] == desarrollador)])
            if todo != 0 or cantidaditems != 0:
                porcentaje = f"{round(cantidaditems*100/todo, 2)}%"
                lista.append([años, cantidaditems, porcentaje])
            else:
                pass
    elif desarrollador not in listadeveloper:
        print('No cuento con los registros de esa empresa en mi base de datos')

    df = pd.DataFrame(lista, columns=["AÑO_LANZAMIENTO", "CANTIDAD DE JUEGOS", "CONTENIDO FREE"])  # Crear un DataFrame a partir del diccionario
    df.columns = ["AÑO", "CANTIDAD DE JUEGOS", "CONTENIDO FREE"]  # Agregar la fila de encabezado
    df = df.sort_values(by='AÑO', ascending=False)
    return df


In [56]:
developer('STAINLESS GAMES LTD', desarrollador)

,AÑO,CANTIDAD DE JUEGOS,CONTENIDO FREE
1,2011,0,0.0%
0,1997,0,0.0%


#### Funcion Gasto

In [13]:
gasto = fb.cargar_datos_parquet('PARQUET\TABLAS UNIDAS\GASTO.parquet')
gasto.head(3)

,PRECIO,ID_USUARIO,AÑO_LANZAMIENTO,RECOMIENDA
0,9.99,BLAZINGTHEAMAZING,2014,True
1,9.99,EXIAEZ,2015,True
2,9.99,MRPFRESH,2011,True


In [22]:
def userdata(usuario):
    """
    Retorna un diccionario con información sobre el usuario especificado.

    Parámetros:
    usuario (str): El identificador del usuario del cual se desea obtener información.

    Retorna:
    dict: Un diccionario con las siguientes claves:
        - "User": El identificador del usuario.
        - "Dinero gastado": La cantidad total de dinero gastado por el usuario en USD, redondeado a 2 decimales.
        - "% de recomendación": El porcentaje de recomendación calculado como el número de elementos recomendados 
        dividido por el total de elementos del usuario, expresado como un entero.
        - "cantidad de items": El número total de elementos del usuario.
    """
    lista = gasto.ID_USUARIO.unique()  # Obtiene una lista de todos los usuarios únicos
    if usuario in lista:  # Verifica si el usuario especificado está en la lista de usuarios
        dfusuario = gasto[(gasto.ID_USUARIO == usuario)]  # Filtra el DataFrame para obtener solo los datos del usuario especificado
        maneyuser = f"{round(dfusuario.PRECIO.sum(),2)} USD"  # Calcula el total gastado por el usuario y lo redondea a 2 decimales
        recomend = f"{int(len(dfusuario[dfusuario.RECOMIENDA == 1])*100/len(dfusuario))}%"  # Calcula el porcentaje de recomendación del usuario
        itemss = len(dfusuario)  # Obtiene el número total de elementos del usuario
        data = {
            "User": usuario,
            "Dinero gastado": maneyuser,
            "% de recomendación": recomend,
            "cantidad de items": itemss
        }
        return data  # Retorna el diccionario con la información del usuario


#### Funcion USUARIO_GENERO

In [34]:
usuario_genero = fb.cargar_datos_parquet('PARQUET\\TABLAS UNIDAS\\USUARIO_GENERO.parquet')
usuario_genero.head(3)

,GENEROS,ID_USUARIO,AÑO_LANZAMIENTO,HORAS_DE_JUEGO
0,ACTION,MRPFRESH,2011,2.32
1,ACTION,ARMOUREDMARSHMALLOW,2014,13.13
2,ACTION,BLUEGILLS,2013,1.07


In [35]:
def UserForGenre(genero):  

    años = usuario_genero.AÑO_LANZAMIENTO.unique()
    lista = []
    df_genero = usuario_genero[usuario_genero.GENEROS == genero]
    usuario = df_genero.groupby('ID_USUARIO')["HORAS_DE_JUEGO"].sum().idxmax()
    for i in años:
        df = df_genero[df_genero.AÑO_LANZAMIENTO == i]
        usuario2 = df[df.ID_USUARIO == usuario]["HORAS_DE_JUEGO"].sum()

        lista.append([i,usuario,usuario2])
    lista_ordenada = sorted(lista, key=lambda x: x[0], reverse=True)
    return lista_ordenada

In [36]:
UserForGenre('RPG')

[[2015, 'COMPANOTHEMUSLIM', 441.63],
 [2014, 'COMPANOTHEMUSLIM', 96.08],
 [2013, 'COMPANOTHEMUSLIM', 34.2],
 [2012, 'COMPANOTHEMUSLIM', 0.0],
 [2011, 'COMPANOTHEMUSLIM', 0.0],
 [2010, 'COMPANOTHEMUSLIM', 0.0]]

#### Funcion TOP_DEV

In [37]:
top_dev = fb.cargar_datos_parquet('PARQUET\TABLAS UNIDAS\TOP_DEV.parquet')
top_dev.head(3)

,DEVELOPER,AÑO,ID_USUARIO,RECOMIENDA,SENTIMIENTO
0,STAINLESS GAMES LTD,1997,BLAZINGTHEAMAZING,True,1
1,VALVE,1998,EXIAEZ,True,1
2,VALVE,1998,MRPFRESH,True,0


In [38]:
def best_developer_year(year):
    rango_aceptado = range(2010, 2016)
    if year not in rango_aceptado:
        return {"message": "Mi base de datos solo tiene registros entre 2010 y 2015"}
    year = int(year)
    recom = top_dev[(top_dev.RECOMIENDA == 1) & (top_dev.SENTIMIENTO == 2) & (top_dev.AÑO == year)]
    developer_recommendations = recom.groupby("DEVELOPER")["RECOMIENDA"].sum()
    top_3_developers = developer_recommendations.nlargest(3)

    top_developers_list = []
    for rank, (developer, count) in enumerate(top_3_developers.items(), start=1):
        top_developers_list.append({"Puesto " + str(rank): developer})

    return top_developers_list

In [40]:
best_developer_year(2010)

[{'Puesto 1': 'FIRAXIS GAMES,ASPYR (MAC, LINUX)'},
 {'Puesto 2': 'TALEWORLDS ENTERTAINMENT'},
 {'Puesto 3': 'OBSIDIAN ENTERTAINMENT'}]

#### Funcion SENT_DEV

In [41]:
sent_dev = fb.cargar_datos_parquet('PARQUET\TABLAS UNIDAS\SENT_DEV.parquet')
sent_dev.head(3)

,DEVELOPER,SENTIMIENTO
0,STAINLESS GAMES LTD,1
1,VALVE,1
2,VALVE,0


In [44]:
def developer_reviews_analysis(desarrolladora):#: str, df_games_and_reviews: pd.DataFrame):
    # Filtramos por desarrolladora
    df_filt_developer = sent_dev[(sent_dev['DEVELOPER'] == desarrolladora) & (sent_dev['SENTIMIENTO'] != 1)]
    # Verificamos que haya datos para la desarrolladora
    if not df_filt_developer.empty:
        # Contamos los sentimientos y mapeamos el número del sentimiento a su etiqueta correspondiente
        sentiment_counts = df_filt_developer['SENTIMIENTO'].value_counts()
        sentiment_mapping = {0: 'Negativo', 2: 'Positivo'}
        sentiment_counts.index = sentiment_counts.index.map(sentiment_mapping)
        
        result = {desarrolladora: sentiment_counts.to_dict()}
    else:
        result = 'No cuento con los registros de esa empresa en mi base de datos'

    return result

In [45]:
developer_reviews_analysis('VALVE')

{'VALVE': {'Positivo': 5344, 'Negativo': 614}}